# Day 5

https://adventofcode.com/2023/day/5

In [1]:
from copy import copy
from dataclasses import dataclass

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import numpy as np

@dataclass
class MappingData():
    destination: int
    source: int
    mrange: int
        
    @property
    def min_source(self):
        return self.source
    
    @property
    def max_source(self):
        return self.source + self.mrange - 1
    
    @property
    def min_destination(self):
        return self.destination
    
    @property
    def max_destination(self):
        return self.destination + self.mrange + 1
    
    @property
    def shift(self):
        return self.destination - self.source
    
    def map(self, i: int) -> int:
        return i + self.shift
    
    def __hash__(self) -> int:
        return hash((self.destination, self.source, self.mrange))
    
    def __repr__(self) -> str:
        return f'({id(self)} {self.min_source}–{self.max_source})->({self.min_destination}-{self.max_destination}) [{self.shift:+}]'
        

class Mapping():
    # First attempt. Realised this is too slow to construct for the real data.
    
    def __init__(self, name, mapping_data):
        self.name = name
        self.mapping = {}
        for md in mapping_data:
            destination, source, mrange = md
            for i in range(mrange):
                self.mapping[source+i] = destination+i
        
    def map(self, i: int) -> int:
        mapping = self.mapping.get(i)
        if mapping:
            return mapping
        else:
            return i
        
class Range:
    
    def __init__(self, lower, upper) -> None:
        self.lower = lower
        self.upper = upper
        
    def __repr__(self) -> str:
        return f"Range: {self.lower} -> {self.upper}"
        
        
class LookupMapping():
    # Generate mappings on the fly.
    
    def __init__(self, name, mapping_data):
        self.name = name
        self.mapping_data = sorted([MappingData(*md) for md in mapping_data],
                                   key=lambda x: x.source)
        
    def map(self, i: int) -> int:
        for md in self.mapping_data:
            if (i >= md.min_source) and (i <= md.max_source):
                return i + md.shift
        return i
    
    def map_ranges(self, ranges: list[Range]) -> list[Range]:
        ranges = copy(ranges)
        mapped_ranges = []
        while ranges:
            r = ranges.pop()
            mp_lower = self.mapping_path(r.lower)
            mp_upper = self.mapping_path(r.upper)
            if mp_lower == mp_upper:
                # if the lower and upper limits of a range follow the same mapping
                # we can map the whole range by mapping the lower and upper limits
                mapped_ranges.append(Range(self.map(r.lower), self.map(r.upper)))
            else: # we need to split the range into two and map these separately
                if mp_lower:
                    # if a mapping exists for the lower limit, map up to the upper source value
                    # for that mapping
                    mapped_ranges.append(Range(self.map(r.lower), self.map(mp_lower.max_source)))
                    # source values out of bounds for this mapping are split off into a new Range
                    # and added back to the stack
                    ranges.append(Range(mp_lower.max_source + 1, r.upper))
                else:
                    # if a mapping does not exist for the lower limit, use the mapping for the
                    # upper limit instead.
                    # map from the lower source value for that mapping to the upper range limit
                    mapped_ranges.append(Range(self.map(mp_upper.min_source), self.map(r.upper)))
                    # source values out of bounds for this mapping are split off into a new Range
                    # and added back to the stack
                    ranges.append(Range(r.lower, mp_upper.min_source - 1))
        return mapped_ranges
        
    
    def __str__(self) -> str:
        to_return = self.name + ':\n'
        for md in self.mapping_data:
            to_return += f'{md.min_source} — {md.max_source} -> {md.min_destination} — {md.max_destination} ({md.shift:+})\n'
        return to_return
    
    def mapping_path(self, i: int) -> int:
        for md in self.mapping_data:
            if (i >= md.min_source) and (i <= md.max_source):
                return md
        return None            


def read_input(filename: str) -> [list[int], list[Mapping]]:
    with open(filename, 'r') as f:
        data = f.read().split('\n\n')
        seeds = list(map(int, data[0].split()[1:]))
        maps = []
        for mapping_data in data[1:]:
            lines = mapping_data.split('\n')
            name = lines[0][:-1]
            mapping_input = [list(map(int, line.split())) for line in lines[1:] if line]
            maps.append(LookupMapping(name=name,
                                      mapping_data=mapping_input))
    return seeds, maps

def read_seed_ranges(filename: str) -> list[Range]:
    ranges = []
    with open(filename, 'r') as f:
        data = list(map(int, f.readline().split()[1:]))
        for i, j in zip(data[::2], data[1::2]):
            ranges.append(Range(i, i+j-1))
    return ranges
    
def seed_to_location(i: int,
                     maps: list[LookupMapping]) -> int:
    for m in maps:
        i = m.map(i)
    return i

def seed_ranges_to_location(ranges: list[Range], 
                            maps: list[LookupMapping]) -> int:
    for m in maps:
        ranges = m.map_ranges(ranges)
    return sorted(ranges, key=lambda x: x.lower)[0].lower

def part2(filename: str) -> int:
    _, maps = read_input(filename)
    ranges = read_seed_ranges(filename)
    return seed_ranges_to_location(ranges=ranges, maps=maps)

def part1(filename: str) -> int:
    seeds, maps = read_input(filename)
    return min(map(lambda x: seed_to_location(x, maps), seeds))

In [2]:
part1('./2021-12-05 AoC example data.txt')

35

In [3]:
part1('./2021-12-05 AoC data.txt')

88151870

In [4]:
part2('2021-12-05 AoC example data.txt')

46

In [5]:
part2('2021-12-05 AoC data.txt')

2008785

In [6]:
%timeit part2('2021-12-05 AoC data.txt')

3.08 ms ± 32.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
